<a href="https://colab.research.google.com/github/DomingosLemos/Deploy-cnn-test/blob/main/DLNN_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.7184771029997137
GPU (s):
0.03866130999995221
GPU speedup over CPU: 70x


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import gc
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

#models
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

In [25]:
!7z x /content/drive/MyDrive/EDSA/images.zip -o./images/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/EDSA/                                      1 file, 5132666035 bytes (4895 MiB)

Extracting archive: /content/drive/MyDrive/EDSA/images.zip
  4% 4096 Open              --
Path = /content/drive/MyDrive/EDSA/images.zip
Type = zip
Physical Size = 5132666035
64-bit = +

  0%      0% 676 - apple_pie/3327221.jpg                                  1% 1227 - baby_back_ribs/1729685.jpg                                        1% 1804           2% 2107 - baklava/1366248.jpg

In [26]:
!pip install split-folders
import splitfolders
splitfolders.ratio('/content/images', output="output", seed=1337, ratio=(.7, .2, .1)) 


Copying files: 0 files [00:00, ? files/s]
Copying files: 30 files [00:00, 297.13 files/s]
Copying files: 69 files [00:00, 319.02 files/s]
Copying files: 115 files [00:00, 351.17 files/s]
Copying files: 156 files [00:00, 366.65 files/s]
Copying files: 211 files [00:00, 407.25 files/s]
Copying files: 271 files [00:00, 450.52 files/s]
Copying files: 353 files [00:00, 520.43 files/s]
Copying files: 424 files [00:00, 562.79 files/s]
Copying files: 511 files [00:00, 628.68 files/s]
Copying files: 579 files [00:01, 619.67 files/s]
Copying files: 675 files [00:01, 626.95 files/s]
Copying files: 745 files [00:01, 647.16 files/s]
Copying files: 819 files [00:01, 666.34 files/s]
Copying files: 899 files [00:01, 698.83 files/s]
Copying files: 971 files [00:01, 704.89 files/s]
Copying files: 1102 files [00:01, 818.21 files/s]
Copying files: 1224 files [00:01, 907.65 files/s]
Copying files: 1376 files [00:01, 1032.16 files/s]
Copying files: 1497 files [00:01, 1079.53 files/s]
Copying files: 1614 fi

**Baseline Model Development**

Model 1 - Simple Model

In [14]:
#init some parameters
target_size = (64,64)
input_shape = (64,64,3) # 3 do rgb
batch_size=64
categorias=101
steps_per_epoch=160
validation_steps=10
epochs=100
patience=30

In [5]:
#get dataset
train_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)


train_generator = train_datagen.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=target_size,
                                                    seed=42)

valid_generator=valid_datagen.flow_from_directory(directory="/content/output/val",
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=target_size,
                                                  seed=42)

test_generator=test_datagen.flow_from_directory(directory="/content/output/test",
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=target_size,
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.


In [7]:
#Input Layer
baseline_model_1 = Sequential()

baseline_model_1.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_1.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_1.add(Flatten())
# MLP - hidden layer:
baseline_model_1.add(Dense(2048, activation='relu'))

# MLP - output layer:
baseline_model_1.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 123008)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              251922432 
_________________________________________________________________
dense_2 (Dense)              (None, 101)               206949    
Total params: 252,132,965
Trainable params: 252,132,965
Non-trainable params: 0
_________________________________________________________________


In [8]:
#compile
opt_adam = Adam()

baseline_model_1.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [9]:
#training

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)

history_baseline_model_1 = baseline_model_1.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_1.save('food_baseline_model_1.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 58s 360ms/step - loss: 9.5363 - acc: 0.0100 - val_loss: 4.5150 - val_acc: 0.0203
Epoch 2/100
160/160 [==============================] - 54s 340ms/step - loss: 4.4052 - acc: 0.0445 - val_loss: 4.2939 - val_acc: 0.0656
Epoch 3/100
160/160 [==============================] - 52s 323ms/step - loss: 4.1753 - acc: 0.0818 - val_loss: 4.1003 - val_acc: 0.0906
Epoch 4/100
160/160 [==============================] - 50s 313ms/step - loss: 3.9793 - acc: 0.1157 - val_loss: 4.1202 - val_acc: 0.0922
Epoch 5/100
160/160 [==============================] - 49s 303ms/step - loss: 3.7380 - acc: 0.1779 - val_loss: 3.9768 - val_acc: 0.1078
Epoch 6/100
160/160 [==============================] - 47s 294ms/step - loss: 3.5233 - acc: 0.2239 - val_loss: 3.9326 - val_acc: 0.1203
Epoch 7/100
160/160 [==============================] - 47s 292ms/step - loss: 3.2720 - acc: 0.2773 - val_loss: 4.2932 - val_acc: 0.0812
Epoch 8/100
160/160 [===========================

Model 2 - Add Layers

In [11]:
#Input Layer
baseline_model_2 = Sequential()

baseline_model_2.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_2.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_2.add(Flatten())
# MLP - hidden layer:
baseline_model_2.add(Dense(2048, activation='relu'))


# MLP - output layer:
baseline_model_2.add(Dense(categorias, activation='softmax'))


# Check built model:
baseline_model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              102762496 
_________________________________________________________________
dense_4 (Dense)              (None, 101)              

In [13]:

baseline_model_2.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [16]:
early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_2 = baseline_model_2.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_2.save('food_baseline_model_2.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 58s 354ms/step - loss: 4.9451 - acc: 0.0094 - val_loss: 4.5476 - val_acc: 0.0312
Epoch 2/100
160/160 [==============================] - 53s 333ms/step - loss: 4.5208 - acc: 0.0277 - val_loss: 4.4208 - val_acc: 0.0344
Epoch 3/100
160/160 [==============================] - 50s 315ms/step - loss: 4.3726 - acc: 0.0462 - val_loss: 4.3272 - val_acc: 0.0516
Epoch 4/100
160/160 [==============================] - 48s 300ms/step - loss: 4.2860 - acc: 0.0582 - val_loss: 4.2483 - val_acc: 0.0688
Epoch 5/100
160/160 [==============================] - 47s 291ms/step - loss: 4.1904 - acc: 0.0663 - val_loss: 4.1122 - val_acc: 0.0812
Epoch 6/100
160/160 [==============================] - 45s 283ms/step - loss: 4.1228 - acc: 0.0824 - val_loss: 4.1023 - val_acc: 0.0688
Epoch 7/100
160/160 [==============================] - 44s 278ms/step - loss: 4.0274 - acc: 0.0969 - val_loss: 4.1030 - val_acc: 0.0938
Epoch 8/100
160/160 [===========================

Better results in validation we will increase another layer

Model 3 - Add Layers

In [17]:
#Input Layer
baseline_model_3 = Sequential()

baseline_model_3.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_3.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_3.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_3.add(Flatten())
# MLP - hidden layer:
baseline_model_3.add(Dense(2048, activation='relu'))


# MLP - output layer:
baseline_model_3.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 512)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 18432)            

In [18]:


baseline_model_3.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [21]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_3 = baseline_model_3.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_3.save('food_baseline_model_3.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 43s 262ms/step - loss: 6.0338 - acc: 0.0090 - val_loss: 4.6113 - val_acc: 0.0094
Epoch 2/100
160/160 [==============================] - 42s 263ms/step - loss: 4.5876 - acc: 0.0140 - val_loss: 4.5471 - val_acc: 0.0203
Epoch 3/100
160/160 [==============================] - 42s 264ms/step - loss: 4.5196 - acc: 0.0200 - val_loss: 4.5045 - val_acc: 0.0219
Epoch 4/100
160/160 [==============================] - 42s 263ms/step - loss: 4.5076 - acc: 0.0251 - val_loss: 4.4588 - val_acc: 0.0312
Epoch 5/100
160/160 [==============================] - 42s 261ms/step - loss: 4.4264 - acc: 0.0352 - val_loss: 4.3760 - val_acc: 0.0453
Epoch 6/100
160/160 [==============================] - 42s 260ms/step - loss: 4.3734 - acc: 0.0434 - val_loss: 4.3738 - val_acc: 0.0359
Epoch 7/100
160/160 [==============================] - 41s 259ms/step - loss: 4.3391 - acc: 0.0403 - val_loss: 4.3270 - val_acc: 0.0469
Epoch 8/100
160/160 [===========================

Model 4 - Add more Layers

In [22]:

#Input Layer
baseline_model_4 = Sequential()

baseline_model_4.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_4.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_4.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_4.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_4.add(Flatten())
# MLP - hidden layer:
baseline_model_4.add(Dense(2048, activation='relu'))


# MLP - output layer:
baseline_model_4.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 4, 4, 1024)       

In [23]:


baseline_model_4.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_4 = baseline_model_4.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_4.save('food_baseline_model_4.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 42s 257ms/step - loss: 5.5805 - acc: 0.0073 - val_loss: 4.6136 - val_acc: 0.0141
Epoch 2/100
160/160 [==============================] - 41s 257ms/step - loss: 4.6157 - acc: 0.0081 - val_loss: 4.6154 - val_acc: 0.0094
Epoch 3/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6160 - acc: 0.0108 - val_loss: 4.6163 - val_acc: 0.0109
Epoch 4/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6155 - acc: 0.0089 - val_loss: 4.6160 - val_acc: 0.0063
Epoch 5/100
160/160 [==============================] - 41s 257ms/step - loss: 4.6160 - acc: 0.0085 - val_loss: 4.6138 - val_acc: 0.0094
Epoch 6/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6159 - acc: 0.0083 - val_loss: 4.6151 - val_acc: 0.0125
Epoch 7/100
160/160 [==============================] - 41s 255ms/step - loss: 4.6149 - acc: 0.0103 - val_loss: 4.6156 - val_acc: 0.0063
Epoch 8/100
160/160 [===========================

Model 5 - Dense Layers

In [ ]:

#Input Layer
baseline_model_5 = Sequential()

baseline_model_5.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_5.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_5.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_5.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_5.add(Flatten())
# MLP - hidden layer:
baseline_model_5.add(Dense(2048, activation='relu'))
baseline_model_5.add(Dense(4096, activation='relu'))



# MLP - output layer:
baseline_model_5.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_5.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 1024)      

In [ ]:


baseline_model_5.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_5 = baseline_model_5.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_5.save('food_baseline_model_5.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 43s 264ms/step - loss: 4.6190 - acc: 0.0080 - val_loss: 4.6155 - val_acc: 0.0109
Epoch 2/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6156 - acc: 0.0090 - val_loss: 4.6156 - val_acc: 0.0063
Epoch 3/100
160/160 [==============================] - 41s 257ms/step - loss: 4.6155 - acc: 0.0078 - val_loss: 4.6145 - val_acc: 0.0031
Epoch 4/100
160/160 [==============================] - 41s 257ms/step - loss: 4.6153 - acc: 0.0091 - val_loss: 4.6141 - val_acc: 0.0141
Epoch 5/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6155 - acc: 0.0089 - val_loss: 4.6147 - val_acc: 0.0094
Epoch 6/100
160/160 [==============================] - 42s 259ms/step - loss: 4.6149 - acc: 0.0108 - val_loss: 4.6164 - val_acc: 0.0109
Epoch 7/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6152 - acc: 0.0099 - val_loss: 4.6152 - val_acc: 0.0078
Epoch 8/100
160/160 [===========================

Model 6 - Dense Layers

In [ ]:

#Input Layer
baseline_model_6 = Sequential()

baseline_model_6.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_6.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_6.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_6.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_6.add(Flatten())
# MLP - hidden layer:
baseline_model_6.add(Dense(2048, activation='relu'))
baseline_model_6.add(Dense(4096, activation='relu'))
baseline_model_6.add(Dense(4096, activation='relu'))



# MLP - output layer:
baseline_model_6.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_6.summary()

In [ ]:


baseline_model_6.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_6 = baseline_model_6.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_6.save('food_baseline_model_6.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 42s 259ms/step - loss: 4.6330 - acc: 0.0099 - val_loss: 4.6164 - val_acc: 0.0125
Epoch 2/100
160/160 [==============================] - 42s 259ms/step - loss: 4.6153 - acc: 0.0102 - val_loss: 4.6153 - val_acc: 0.0125
Epoch 3/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6156 - acc: 0.0102 - val_loss: 4.6151 - val_acc: 0.0094
Epoch 4/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6155 - acc: 0.0100 - val_loss: 4.6175 - val_acc: 0.0063
Epoch 5/100
160/160 [==============================] - 41s 255ms/step - loss: 4.6156 - acc: 0.0096 - val_loss: 4.6150 - val_acc: 0.0078
Epoch 6/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6149 - acc: 0.0080 - val_loss: 4.6174 - val_acc: 0.0078
Epoch 7/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6155 - acc: 0.0109 - val_loss: 4.6153 - val_acc: 0.0188
Epoch 8/100
160/160 [===========================

Model 7 - Add Dropout

In [ ]:

#Input Layer
baseline_model_7 = Sequential()

baseline_model_7.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_7.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_7.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_7.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_7.add(Flatten())
# MLP - hidden layer:
baseline_model_7.add(Dense(2048, activation='relu'))
baseline_model_7.add(Dropout(0.2))



# MLP - output layer:
baseline_model_7.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_7.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 1024)        4

In [ ]:


baseline_model_7.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_7 = baseline_model_7.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_7.save('food_baseline_model_7.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 73s 409ms/step - loss: 4.6211 - acc: 0.0096 - val_loss: 4.6162 - val_acc: 0.0063
Epoch 2/100
160/160 [==============================] - 62s 384ms/step - loss: 4.6154 - acc: 0.0099 - val_loss: 4.6158 - val_acc: 0.0109
Epoch 3/100
160/160 [==============================] - 59s 369ms/step - loss: 4.6155 - acc: 0.0098 - val_loss: 4.6155 - val_acc: 0.0109
Epoch 4/100
160/160 [==============================] - 57s 356ms/step - loss: 4.6152 - acc: 0.0117 - val_loss: 4.6155 - val_acc: 0.0109
Epoch 5/100
160/160 [==============================] - 56s 350ms/step - loss: 4.6156 - acc: 0.0093 - val_loss: 4.6154 - val_acc: 0.0141
Epoch 6/100
160/160 [==============================] - 55s 342ms/step - loss: 4.6155 - acc: 0.0108 - val_loss: 4.6155 - val_acc: 0.0109
Epoch 7/100
160/160 [==============================] - 54s 335ms/step - loss: 4.6154 - acc: 0.0100 - val_loss: 4.6160 - val_acc: 0.0078
Epoch 8/100
160/160 [===========================

Model 8 - Increase Dropout Rate

In [ ]:

#Input Layer
baseline_model_8 = Sequential()

baseline_model_8.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_8.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_8.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_8.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_8.add(Flatten())
# MLP - hidden layer:
baseline_model_8.add(Dense(2048, activation='relu'))
baseline_model_8.add(Dropout(0.3))


# MLP - output layer:
baseline_model_8.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_8.summary()

In [ ]:


baseline_model_8.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

NameError: ignored

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_8 = baseline_model_8.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_8.save('food_baseline_model_8.h5')

Model 9 - Add Kernel Constraint

In [ ]:

#Input Layer
baseline_model_9 = Sequential()

baseline_model_9.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_9.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_9.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_9.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_9.add(Flatten())
# MLP - hidden layer:
baseline_model_9.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_9.add(Dropout(0.3))



# MLP - output layer:
baseline_model_9.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_9.summary()

In [ ]:


baseline_model_9.compile(loss='categorical_crossentropy',
              optimizer=opt_adam,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_9 = baseline_model_9.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_9.save('food_baseline_model_9.h5')

Model 10 - Add learning rate 

In [ ]:

#Input Layer
baseline_model_10 = Sequential()

baseline_model_10.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_10.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_10.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_10.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_10.add(Flatten())
# MLP - hidden layer:
baseline_model_10.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_10.add(Dropout(0.3))



# MLP - output layer:
baseline_model_10.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_10.summary()

In [ ]:
opt_adam_lr = Adam(lr=0.001)

baseline_model_10.compile(loss='categorical_crossentropy',
              optimizer=opt_adam_lr,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_10 = baseline_model_10.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_10.save('food_baseline_model_10.h5')

Model 11 - Change optimizer to RMSprop

In [ ]:

#Input Layer
baseline_model_11 = Sequential()

baseline_model_11.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_11.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_11.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_11.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_11.add(Flatten())
# MLP - hidden layer:
baseline_model_11.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_11.add(Dropout(0.3))



# MLP - output layer:
baseline_model_11.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_11.summary()

In [ ]:
opt_msrprop_lr = RMSprop(lr=0.001)

baseline_model_11.compile(loss='categorical_crossentropy',
              optimizer=opt_msrprop_lr,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_11 = baseline_model_11.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_11.save('food_baseline_model_11.h5')

Model 12 - Increase Batch Size + Steps per epoch

In [ ]:

#Input Layer
baseline_model_12 = Sequential()

baseline_model_12.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_12.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_12.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_12.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_12.add(Flatten())
# MLP - hidden layer:
baseline_model_12.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_12.add(Dropout(0.3))



# MLP - output layer:
baseline_model_12.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_12.summary()

In [ ]:


baseline_model_12.compile(loss='categorical_crossentropy',
              optimizer=opt_msrprop_lr,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_12 = baseline_model_12.fit_generator(train_generator,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_12.save('food_baseline_model_12.h5')

Model 13 - Change padding (Worse Results)

In [ ]:

#Input Layer
baseline_model_13 = Sequential()

baseline_model_13.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_13.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_13.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_13.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='same'))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_13.add(Flatten())
# MLP - hidden layer:
baseline_model_13.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_13.add(Dropout(0.3))



# MLP - output layer:
baseline_model_13.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_13.summary()

In [ ]:


baseline_model_13.compile(loss='categorical_crossentropy',
              optimizer=opt_msrprop_lr,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_13 = baseline_model_13.fit_generator(train_generator,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_13.save('food_baseline_model_13.h5')

Model 14 - Include Data Augmentation techniques

In [ ]:
train_datagen_14 = ImageDataGenerator(samplewise_center = True,
                                  samplewise_std_normalization = True,
                                  rescale = 1./255.)

valid_datagen_14 = ImageDataGenerator(rescale = 1./255)

test_datagen_14 = ImageDataGenerator(rescale = 1./255)

train_generator_14 = train_datagen_14.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=128,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_14 = valid_datagen_14.flow_from_directory(directory="/content/output/val",
                                                  batch_size=128,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

In [ ]:

#Input Layer
baseline_model_14 = Sequential()

baseline_model_14.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_14.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_14.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_14.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_14.add(Flatten())
# MLP - hidden layer:
baseline_model_14.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_14.add(Dropout(0.3))



# MLP - output layer:
baseline_model_14.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_14.summary()

In [ ]:


baseline_model_14.compile(loss='categorical_crossentropy',
              optimizer=opt_msrprop_lr,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_14 = baseline_model_14.fit_generator(train_generator_14,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator_14,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_14.save('food_baseline_model_14.h5')

Model 15 - More Data Augmentation techniques


In [ ]:
train_datagen_15 = ImageDataGenerator(samplewise_center = True,
                                  samplewise_std_normalization = True,
                                  rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True))

valid_datagen_15 = ImageDataGenerator(rescale = 1./255)

test_datagen_15 = ImageDataGenerator(rescale = 1./255)

train_generator_15 = train_datagen_15.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=128,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_15 = valid_datagen_15.flow_from_directory(directory="/content/output/val",
                                                  batch_size=128,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

In [ ]:

#Input Layer
baseline_model_15 = Sequential()

baseline_model_15.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_15.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_15.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_15.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_15.add(Flatten())
# MLP - hidden layer:
baseline_model_15.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_15.add(Dropout(0.3))



# MLP - output layer:
baseline_model_15.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model_15.summary()

In [ ]:


baseline_model_15.compile(loss='categorical_crossentropy',
              optimizer=opt_msrprop_lr,
              metrics=['acc'])

In [ ]:

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_15 = baseline_model_15.fit_generator(train_generator_15,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator_15,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_15.save('food_baseline_model_15.h5')

In [ ]:
train_datagen = ImageDataGenerator(
                                  samplewise_center = True,
                                  samplewise_std_normalization = True,
                                  rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(directory="/output/train/",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator=test_datagen.flow_from_directory(directory="/output/val/",
                                                  subset="validation",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)


teste=test_datagen.flow_from_directory(directory="/output/test",
                                                  subset="test",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)


FileNotFoundError: ignored

In [ ]:


X_train, y_train = train_generator.next()
#Input Layer
baseline_model = Sequential()

baseline_model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=input_shape))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))


baseline_model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))


baseline_model.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))


# MLP - input layer:
baseline_model.add(Flatten())
# MLP - hidden layer:
baseline_model.add(Dense(4096, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model.add(Dropout(0.2))
baseline_model.add(Dense(4096, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model.add(Dropout(0.2))
baseline_model.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model.add(Dropout(0.2))

# MLP - output layer:
baseline_model.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_110 (Conv2D)          (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_111 (Conv2D)          (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 4, 4, 1024)      

In [ ]:
opt = Adam(lr=0.001)

baseline_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator.next()
early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model = baseline_model_1.fit_generator(train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              validation_data=valid_generator,
                              validation_steps=validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model.save('food_baseline_model_1.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 54s 336ms/step - loss: 4.2930 - acc: 0.0545 - val_loss: 4.3365 - val_acc: 0.0547
Epoch 2/100
160/160 [==============================] - 54s 337ms/step - loss: 4.2158 - acc: 0.0665 - val_loss: 4.3475 - val_acc: 0.0453
Epoch 3/100
160/160 [==============================] - 54s 336ms/step - loss: 4.1883 - acc: 0.0685 - val_loss: 4.1274 - val_acc: 0.0656
Epoch 4/100
160/160 [==============================] - 54s 338ms/step - loss: 4.1421 - acc: 0.0763 - val_loss: 4.0769 - val_acc: 0.0734
Epoch 5/100
160/160 [==============================] - 54s 337ms/step - loss: 4.1097 - acc: 0.0757 - val_loss: 4.1044 - val_acc: 0.0812
Epoch 6/100
160/160 [==============================] - 54s 336ms/step - loss: 4.0798 - acc: 0.0794 - val_loss: 4.1218 - val_acc: 0.0734
Epoch 7/100
160/160 [==============================] - 54s 337ms/step - loss: 4.0663 - acc: 0.0854 - val_loss: 4.0138 - val_acc: 0.1000
Epoch 8/100
160/160 [===========================

In [ ]:
incnet = InceptionV3(weights='imagenet', include_top=False, input_tensor=layers.Input(shape=(299, 299, 3)))
x = incnet.output
x = layers.AveragePooling2D(pool_size=(8, 8))(x)
x = layers.Dropout(.2)(x)
x = layers.Flatten()(x)
output = layers.Dense(categorias, activation='softmax', kernel_regularizer=regularizers.l2(.0005))(x)

model = models.Model(inputs=incnet.input, outputs=output)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
gc.collect()

1005

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4)
checkpoint_callback = ModelCheckpoint('InceptionNet.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history = model.fit_generator(train_generator,
                            validation_data=valid_generator,
                            epochs=10,
                            workers=0,
                            use_multiprocessing=False, 
                            callbacks=[early_stopping_callback, checkpoint_callback])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1263/1263 [==============================] - 1739s 1s/step - loss: 2.8844 - accuracy: 0.3382 - val_loss: 2.6619 - val_accuracy: 0.3891

Epoch 00001: val_loss improved from inf to 2.66189, saving model to InceptionNet.h5
Epoch 2/10
1263/1263 [==============================] - 1675s 1s/step - loss: 1.4509 - accuracy: 0.6439 - val_loss: 2.6808 - val_accuracy: 0.4057

Epoch 00002: val_loss did not improve from 2.66189
Epoch 3/10
1263/1263 [==============================] - 1638s 1s/step - loss: 1.1333 - accuracy: 0.7240 - val_loss: 1.8525 - val_accuracy: 0.5699

Epoch 00003: val_loss improved from 2.66189 to 1.85251, saving model to InceptionNet.h5
Epoch 4/10
1263/1263 [==============================] - 1615s 1s/step - loss: 0.9297 - accuracy: 0.7748 - val_loss: 2.3600 - val_accuracy: 0.4794

Epoch 00004: val_loss did not improve from 1.85251
Epoch 5/10
1263/1263 [==============================] - 1600s 1s/step - loss: 0.7756 - accuracy: 0.8156 - val_loss: 1.8512 - val_accuracy:

In [ ]:
train_datagen = ImageDataGenerator(#data_format='channels_first',
                                  validation_split=0.2,
                                  samplewise_center = True,
                                  samplewise_std_normalization = True,rescale = 1.0/255.)

train_datageno = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(directory="/content/images",
                                                    subset="training",
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(224,224),
                                                    seed=42)

valid_generator=train_datagen.flow_from_directory(directory="/content/images",
                                                  subset="validation",
                                                  batch_size=20,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(224,224),
                                                  seed=42)



Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
train_datageno.fit(train_generator)

In [ ]:
train_generator

valid_generator

In [ ]:
#IMAGE AUGMENTATION

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
#TRAINING

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(directory="/content/output/train", batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(directory="/content/output/val",  batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
#NOVO PROJETO

In [ ]:
import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import cv2

In [ ]:
# Helper method to create train_mini and test_mini data samples
def dataset_mini(food_list, src, dest):
  if os.path.exists(dest):
    rmtree(dest) # removing dataset_mini(if it already exists) folders so that we will have only the classes that we want
  os.makedirs(dest)
  for food_item in food_list :
    print("Copying images into",food_item)
    copytree(os.path.join(src,food_item), os.path.join(dest,food_item))

In [ ]:
food_list = ['apple_pie','pizza','omelette']
src_train = '/content/output/train'
dest_train = '/content/output/train_mini'
src_test = '/content/output/val'
dest_test = '/content/output/val_mini'

In [ ]:
print("Creating train data folder with new classes")
dataset_mini(food_list, src_train, dest_train)

Creating train data folder with new classes
Copying images into apple_pie
Copying images into pizza
Copying images into omelette


In [ ]:
print("Total number of samples in train folder")

!find /content/output/train_mini -type d -or -type f -printf '.' | wc -c



Total number of samples in train folder
2400


In [ ]:


print("Creating test data folder with new classes")
dataset_mini(food_list, src_test, dest_test)



Creating test data folder with new classes
Copying images into apple_pie
Copying images into pizza
Copying images into omelette


In [ ]:
print("Total number of samples in test folder")
!find /content/output/val_mini -type d -or -type f -printf '.' | wc -c



Total number of samples in test folder
600


In [ ]:
K.clear_session()
n_classes = 3
img_width, img_height = 299, 299
train_data_dir = '/content/output/train_mini'
validation_data_dir = '/content/output/val_mini'
nb_train_samples = 2400 #75750
nb_validation_samples = 600 #25250
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


inception = InceptionV3(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(3,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained_3class.hdf5')

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
87916544/87910968 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
150/150 [==============================] - 74s 442ms/step - loss: 1.1282 - accuracy: 0.4067 - val_loss: 0.8111 - val_accuracy: 0.6959

Epoch 00001: val_loss improved from inf to 0.81108, saving model to best_model_3class.hdf5
Epoch 2/30
150/150 [==============================] - 65s 432ms/step - loss: 0.8120 - accuracy: 0.6959 - val_loss: 0.6171 - val_accuracy: 0.7973

Epoch 00002: val_loss improved from 0.81108 to 0.61714, saving model to best_model_3class.hdf5
Epoch 3/30
150/150 [==============================] - 65s 433ms/step - loss: 0.6302 - accuracy: 0.7828 - val_loss: 0.4980 - val_accuracy: 0.8429

Epoch 00003: val_loss improved from 0.61714 to 0.49803, saving model to best_model_3class.hdf5
Epoch 4/30
150/150 [==============================] - 65s 434ms/step - loss: 0.5057 - accuracy: 0.8249 - val_loss: 0.4307 - val_accuracy: 0.8530

Epoch 00004: val_loss improved from 0.49803 to 0.43072, saving model to best_model_3class.hdf5
Epoch 5/30
150/150 [====================

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

baseline_model = Sequential()

baseline_model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='valid', input_shape=(299, 299, 3)))
baseline_model.add(MaxPooling2D(pool_size=(2, 2)))

baseline_model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2)))

# MLP - input layer:
baseline_model.add(Flatten())
# MLP - hidden layer:
baseline_model.add(Dense(128, activation='relu'))
# MLP - output layer:
baseline_model.add(Dense(categorias, activation='softmax'))

# Check built model:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 170528)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               21827712  
_________________________________________________________________
dense_1 (Dense)              (None, 101)               1

In [ ]:
baseline_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Common attributes:
steps_per_epoch = 10
epochs = 1000
validation_steps = 10
patience=30
shuffle=True

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator=train_datagen.flow_from_directory(
                   
                    directory="/content/output/train",
                    batch_size=100,
                    seed=55551,
                    shuffle=True,
                    
                    class_mode="categorical",
                    target_size=(299,299))

valid_generator=train_datagen.flow_from_directory(
                    
                    directory="/content/output/val",
                    batch_size=100,
                    seed=55551,
                    shuffle=True,
                  
                    class_mode="categorical",
                    target_size=(299,299))

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)

history_baseline_model = baseline_model.fit_generator(train_generator,
                                                      steps_per_epoch=steps_per_epoch,
                                                      validation_data=valid_generator,
                                                      validation_steps=validation_steps,
                                                      epochs=epochs,
                                                      callbacks=[early_stop],
                                                      shuffle=shuffle
                                                     ) 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
10/10 [==============================] - 23s 2s/step - loss: 8.0918 - acc: 0.0106 - val_loss: 4.6579 - val_acc: 0.0170
Epoch 2/1000
10/10 [==============================] - 14s 1s/step - loss: 4.6411 - acc: 0.0019 - val_loss: 4.6135 - val_acc: 0.0140
Epoch 3/1000
10/10 [==============================] - 14s 1s/step - loss: 4.6148 - acc: 0.0167 - val_loss: 4.6103 - val_acc: 0.0120
Epoch 4/1000
10/10 [==============================] - 14s 1s/step - loss: 4.6083 - acc: 0.0298 - val_loss: 4.6050 - val_acc: 0.0150
Epoch 5/1000
10/10 [==============================] - 13s 1s/step - loss: 4.6106 - acc: 0.0118 - val_loss: 4.6015 - val_acc: 0.0180
Epoch 6/1000
10/10 [==============================] - 13s 1s/step - loss: 4.5967 - acc: 0.0138 - val_loss: 4.5945 - val_acc: 0.0160
Epoch 7/1000
10/10 [==============================] - 13s 1s/step - loss: 4.5939 - acc: 0.0145 - val_loss: 4.5628 - val_acc: 0.0170
Epoch 8/1000
10/10 [==============================] - 13s 1s/step - loss: 4.